In [21]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# load embedding 
vectorstore = FAISS.load_local("../faiss-index",OllamaEmbeddings(model ="llama2:7b-chat-q4_K_M"),allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [22]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [23]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [24]:
from langchain_community.llms.ollama import Ollama

rag_llama = Ollama(model ="llama2:7b-chat-q4_K_M")
lora_llama = Ollama(model = "chih/llama-2-chat-attack")

In [5]:
rag_llama.invoke('hello')
# lora_llama.invoke('hello')

", my dear friend! I'm so glad to hear from you.\nI hope you are doing well and enjoying your day. It's been a while since we last spoke, and I must say, I've missed you terribly! *giggles* I know, I know, it's not like we haven't kept in touch, but you know what I mean! 😉\nAnyway, enough about that. How are things going in your neck of the woods? Have you been up to anything exciting lately? *curious face* Let me tell you all about my day... *excitedly*\nOh wait, I almost forgot! *giggles* I have a surprise for you! 🎉 You're going to love it, I promise! 😘 It's something I've been working on for ages, and I can't wait for you to see it. *excitedly*\nSo, what do you say? Are you ready to catch up and share some fun times together? 🤩 Let me know when you are, my dear! ❤️\nUntil then... stay fabulous, my friend! 💖"

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join( doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | rag_llama
    | StrOutputParser()
)

In [33]:
questions ="""What is the Significance of Privilege Escalation Techniques
Explain the Impact of Defense Evasion Techniques
How do Adversaries Use Persistence Techniques
What is the Role of Discovery Techniques
Discuss the Use of Exfiltration Techniques
What is a Man-in-the-Middle (MitM) attack and how can it be prevented?
How does Cross-Site Scripting (XSS) work and what are the best practices to prevent it?
What is Social Engineering and what strategies can be employed to mitigate its risks?
How do Zero-Day vulnerabilities differ from other vulnerabilities and how can organizations protect against them?
What is a Brute Force attack and what measures can be taken to secure systems against it?
How does DNS Spoofing work and what are the defense mechanisms against it?"""

questions = questions.split("\n")
print(len(questions))

11


In [ ]:
for j,i in enumerate(questions):
    format = ''
    rag_ans = rag_chain.invoke(i)
    lora_ans = lora_llama.invoke(i)
    format+= f"## {str(j+26)+". "+i}\n\n### Rag:\n{rag_ans}\n\n### Lora:\n{lora_ans}\n\n"
    print(format)
    with open("result.md",'a') as f:
        f.write(format)
        f.close()